In [1]:
# -------------------------------------------------------------
# 0. Install requirements (only missing packages)
# -------------------------------------------------------------
import os
import sys
import pkg_resources
import subprocess

print(">> Verifica requirements...")
req_file = 'requirements.txt'
missing = []
if os.path.exists(req_file):
    with open(req_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            try:
                pkg_resources.require(line)
            except Exception:
                missing.append(line)
else:
    # fallback: ensure at least one project package is present
    try:
        pkg_resources.require('ood-metrics')
    except Exception:
        missing.append('ood-metrics')

if missing:
    print(f">> Mancano pacchetti: {missing}. Installazione in corso...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])
else:
    print(">> Tutti i requirements sono già installati.")


In [2]:
REPO_URL = "https://github.com/MatteoSanna23/AnomalySegmentation.git"
REPO_DIR = "AnomalySegmentation"
DATASET_DIR = "tiny-imagenet/tiny-imagenet-200"
ZIP_PATH = "tiny-imagenet-200.zip"

In [3]:
import os

# -------------------------------------------------------------
# 1. Clone repo if not exist
# -------------------------------------------------------------
if not os.path.exists(REPO_DIR):
    print(">> Clonazione della repository...")
    os.system(f"git clone --branch firts_steps {REPO_URL}")  # clone the 'firts_steps' branch
else:
    print(">> Repository già presente, niente clone.")

os.chdir(REPO_DIR)
print(">> Current working dir:", os.getcwd())


In [4]:
# Mount Google Drive if running in Colab; otherwise skip (local/VSCode).
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
    print(">> Google Drive mounted (Colab).")
except Exception:
    print(">> Not running in Colab or google.colab not available. Skipping drive mount.")


In [7]:
# Estrai il file zip se presente (portabile su Windows/Linux)
import os, zipfile
candidates = 'Validation_Dataset.zip'

out_dir = os.path.join(os.getcwd(), 'dataset_validation')
os.makedirs(out_dir, exist_ok=True)
print(">> Estraggo:", candidates, "->", out_dir)
with zipfile.ZipFile(candidates, 'r') as zf:
        zf.extractall(out_dir)
print(">> Estrazione completata.")


In [ ]:
# Verifica di essere nel posto giusto (dovresti vedere evalAnomaly.py)
import os
print(">> Current working dir:", os.getcwd())
print(">> Files:", os.listdir('.'))


In [ ]:
# Pull latest changes in the current repo (if available)
import subprocess
try:
    subprocess.run(['git', 'pull'], check=True)
    print(">> git pull OK")
except Exception as e:
    print(">> git pull failed or not available:", e)


In [ ]:
# Run evaluations using local Python (adjust dataset/weights paths if needed)
import os, sys, subprocess, glob
cwd = os.getcwd()
# look for dataset folder produced by unzip
dataset_root = os.path.join(cwd, '..', 'dataset_validation')
if not os.path.exists(dataset_root):
    dataset_root = os.path.join(cwd, 'dataset_validation')
# locate weights in repo root or known filename
weights_candidates = [os.path.join('..', 'epoch_106-step_19902_eomt.ckpt'), os.path.join(cwd, '..', 'epoch_106-step_19902_eomt.ckpt'), 'epoch_106-step_19902_eomt.ckpt']
weights_path = None
for p in weights_candidates:
    if p and os.path.exists(p):
        weights_path = p
        break
if not weights_path:
    print(">> Checkpoint non trovato nelle posizioni comuni. Aggiorna 'weights_path' se necessario.")

runs = [
    ('ROAD ANOMALY', os.path.join(dataset_root, 'Validation_Dataset', 'RoadAnomaly', 'images', '*.jpg')),
    ('SMIYC RA-21', os.path.join(dataset_root, 'Validation_Dataset', 'RoadAnomaly21', 'images', '*.png')),
    ('ROAD OBSTACLE', os.path.join(dataset_root, 'Validation_Dataset', 'RoadObsticle21', 'images', '*.webp')),
    ('FS L&F', os.path.join(dataset_root, 'Validation_Dataset', 'FS_LostFound_full', 'images', '*.png')),
    ('FS STATIC', os.path.join(dataset_root, 'Validation_Dataset', 'fs_static', 'images', '*.jpg')),
]
for name, pattern in runs:
    print('>>> Esecuzione su', name, '->', pattern)
    files = glob.glob(pattern)
    if not files:
        print('>>> Nessun file trovato per pattern:', pattern); continue
    if not weights_path:
        print('>>> Skipping run: no weights available.')
        break
    cmd = [sys.executable, 'evalAnomaly_temperature.py', '--input', pattern, '--loadWeights', weights_path]
    print('>>> Running:', ' '.join(cmd))
    try:
        subprocess.run(cmd, check=True)
    except Exception as e:
        print('>>> Run failed:', e)


In [ ]:
"""
# Example runs (commented shell commands converted to comments)
# 1. Road Anomaly
# >>> Esecuzione su ROAD ANOMALY...
# !python evalAnomaly_eomt.py --input "/content/dataset_validation/Validation_Dataset/RoadAnomaly/images/*.jpg" --loadWeights "/content/drive/MyDrive/Colab Notebooks/epoch_106-step_19902_eomt.ckpt"

# 2. Road Anomaly (SMIYC RA-21)
# >>> Esecuzione su SMIYC RA-21...
# !python evalAnomaly_eomt.py --input "/content/dataset_validation/Validation_Dataset/RoadAnomaly21/images/*.png" --loadWeights "/content/drive/MyDrive/Colab Notebooks/epoch_106-step_19902_eomt.ckpt"

# 3. Road Obstacle (SMIYC RO-21)
# >>> Esecuzione su ROAD OBSTACLE...
# !python evalAnomaly_eomt.py --input "/content/dataset_validation/Validation_Dataset/RoadObsticle21/images/*.webp" --loadWeights "/content/drive/MyDrive/Colab Notebooks/epoch_106-step_19902_eomt.ckpt"

# 4. Fishyscapes Lost & Found (FS L&F)
# >>> Esecuzione su FS L&F...
# !python evalAnomaly_eomt.py --input "/content/dataset_validation/Validation_Dataset/FS_LostFound_full/images/*.png" --loadWeights "/content/drive/MyDrive/Colab Notebooks/epoch_106-step_19902_eomt.ckpt"

# 5. Fishyscapes Static
# >>> Esecuzione su FS STATIC...
# !python evalAnomaly_eomt.py --input "/content/dataset_validation/Validation_Dataset/fs_static/images/*.jpg" --loadWeights "/content/drive/MyDrive/Colab Notebooks/epoch_106-step_19902_eomt.ckpt"
"""


In [ ]:
"""
# Example runs (commented shell commands converted to comments)
# 1. Road Anomaly
# >>> Esecuzione su ROAD ANOMALY...
# !python evalAnomaly.py --input "/content/dataset_validation/Validation_Dataset/RoadAnomaly/images/*.jpg" --loadWeights "erfnet_pretrained.pth"

# 2. Road Anomaly (SMIYC RA-21)
# >>> Esecuzione su SMIYC RA-21...
# !python evalAnomaly.py --input "/content/dataset_validation/Validation_Dataset/RoadAnomaly21/images/*.png" --loadWeights "erfnet_pretrained.pth"

# 3. Road Obstacle (SMIYC RO-21)
# >>> Esecuzione su ROAD OBSTACLE...
# !python evalAnomaly.py --input "/content/dataset_validation/Validation_Dataset/RoadObsticle21/images/*.webp" --loadWeights "erfnet_pretrained.pth"

# 4. Fishyscapes Lost & Found (FS L&F)
# >>> Esecuzione su FS L&F...
# !python evalAnomaly.py --input "/content/dataset_validation/Validation_Dataset/FS_LostFound_full/images/*.png" --loadWeights "erfnet_pretrained.pth"

# 5. Fishyscapes Static
# >>> Esecuzione su FS STATIC...
# !python evalAnomaly.py --input "/content/dataset_validation/Validation_Dataset/fs_static/images/*.jpg" --loadWeights "erfnet_pretrained.pth"
"""